In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [28]:
url_inicial = 'http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-2008'
url_root = 'http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-2008'
r=requests.get(url_inicial)

In [29]:
soup = BeautifulSoup(r.text, 'html.parser')

In [30]:
box = soup.find('div', class_='sumarios decada-2008')
volumen=box.findAll('div',class_='sumario row')
vol = [x.find('a').get('href')for x in volumen] 
vol = [urljoin (url_root, i) for i in vol]
vol 

['http://www.revistagastroenterologiamexico.org/es-vol-73-num-4-sumario-X0375090608X8211X',
 'http://www.revistagastroenterologiamexico.org/es-vol-73-num-3-sumario-X0375090608X71829',
 'http://www.revistagastroenterologiamexico.org/es-vol-73-num-2-sumario-X0375090608X71927',
 'http://www.revistagastroenterologiamexico.org/es-vol-73-num-1-sumario-X0375090608X71747']

In [31]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='collFull') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1 = [urljoin (url_root, i) for i in vol1]
    vol2 = [urljoin (url_root, i) for i in vol2]
    vol2+=vol1

In [32]:
vol2

['http://www.revistagastroenterologiamexico.org/es-editorial-articulo-X037509060849262X',
 'http://www.revistagastroenterologiamexico.org/es-editorial-articulo-X0375090608497800',
 'http://www.revistagastroenterologiamexico.org/es-editorial-articulo-X0375090608497819',
 'http://www.revistagastroenterologiamexico.org/es-neoplasias-malignas-del-aparato-digestivo-articulo-X0375090608497827',
 'http://www.revistagastroenterologiamexico.org/es-cirugia-colorrectal-laparoscopica-experiencia-un-articulo-X0375090608497835',
 'http://www.revistagastroenterologiamexico.org/es-el-papel-del-abuso-fisico-articulo-X0375090608497843',
 'http://www.revistagastroenterologiamexico.org/es-conceptos-actuales-acerca-del-tratamiento-articulo-X0375090608497851',
 'http://www.revistagastroenterologiamexico.org/es-ileo-mixedematoso-exacerbante-abdomen-agudo-articulo-X037509060849786X',
 'http://www.revistagastroenterologiamexico.org/es-fistula-bilioenterica-con-impactacion-lito-articulo-X0375090608497878',
 'ht

In [33]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='collFull') 
    volumen1=box1.findAll('div', class_='item')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [34]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina http://www.revistagastroenterologiamexico.org/es-numeros-anteriores-2008


In [35]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

78

In [36]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [37]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista de Gastroenterología de México'
    a='III. Medicina y Ciencias de la Salud'
    tem='Gastroenterología'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('div', class_='elsevierItemTitulo').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='elsevierItemTextoCompletoTexto').get_text(strip=True)
        content_book['Resumen']=resu.replace("�� Caso clínico","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
        linkart = [urljoin (url_root, i) for i in linkart]
    #link articulo
    try:
        link=s_item.find('div', class_='right').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
        link = [urljoin (url_root, i) for i in link]
    return content_book

In [38]:
list_scraper=list_scraper[0:101]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [39]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Editorial,En este número de laRevista de Gastroenterolog...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
1,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Editorial,"Boca del Río, Veracruz, 20 de noviembre de 200...",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
2,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Editorial,"""Lo que bien se hace, bien acaba""Quiero expres...",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
3,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Neoplasias malignas del aparato digestivo en p...,IntroducciónEn los últimos 60 años ha habido u...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
4,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Cirugía colorrectal laparoscópica: experiencia...,IntroducciónLa primera cirugía laparoscópica p...,None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
...,...,...,...,...,...,...,...,...
73,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Terlipresina mejora la función renal de pacien...,"Krag A, Møller S, Henriksen JH et al. Terlipre...",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
74,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,"Estudio multicéntrico de 4 semanas, doble cieg...","Johanson JF, Morton D, Geenen J et al. Multice...",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
75,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,Alta incidencia de diabetes mellitus tipo I du...,"Schreuder TCMA, Gelderbrom HC, Weegink CJ et a...",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...
76,Revista de Gastroenterología de México,III. Medicina y Ciencias de la Salud,Gastroenterología,La medición de la elasticidad hepática en niño...,"De Lédinghen V, Le Bail B, Rebouissoux L et al...",None,http://www.revistagastroenterologiamexico.org/...,http://www.revistagastroenterologiamexico.org/...


In [40]:
df_catalogo.to_csv('Revista019.15.csv', index=False)

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 1409-1410: surrogates not allowed